<a href="https://colab.research.google.com/github/wooje-jung/23-2-AI/blob/main/SIMCSE%EC%84%B1%EB%8A%A5%EB%B9%84%EA%B5%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 구글 드라이브 코랩에 마운트하기


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. 훈련데이터 가져오기


In [2]:
import pandas as pd

file_path = "/content/drive/MyDrive/korsts.xlsx"
df = pd.read_excel(file_path)
df = df.dropna()
df
df = df.drop(columns=["Unnamed: 3"])


In [3]:
df.reset_index(drop=True, inplace=True)
df

,label,sentence1,sentence2
0,4.2,이란은 또한 NPT와 IAEA의 서명을 받았습니다.,이란은 또한 NPT와 IAEA의 서명인 EC 팩토이드를 사랑해야 합니다.\n\nma...
1,4.4,"브랜데이스의 관계자들은 이것이 ""매우 가슴 아픈"" 캠퍼스의 시간이라고 말했다.",이번은 브랜데이스 대학 전체 지역사회에 매우 가슴 아픈 시기다.\n\nmain-ne...
2,3.2,"그러나 아리 플라이셔 백악관 대변인은 어제 ""이란 사람들이 알 카 아다를 체포하는 ...",이란 인들이 알 카에다를 체포하는 측면에서 취했다고 주장하는 조치는 불충분합니다.\...
3,2.8,"하지만 나는 낮은 기준보다는 높은 기준에 대해 이야기하고 싶다.""",나는 오히려 부정적인 것보다 긍정적인 숫자에 대해 이야기하고 싶다.\n\nmain-...
4,4.6,"마리아 쳉 대변인은 ""이는 고립된 사건과 매우 흡사하다""고 말했다.",세계 보건기구 대변인 마리아 쳉은 고립된 사건과 매우 흡사합니다.\n\nmain-n...
5,3.2,"아마도, 노년은 우리가 우리의 강점과 약점을 이해할 수 있도록 도와줍니다.","아마도, 노년기는 우리의 힘과 약점과 세계의 현실을 이해하는 데 도움이 됩니다.\n..."
6,4.8,나는 내가 최고의 테니스를 치는 경우에만 이것을 할 수 있다고 믿고 법원에 갈 것이다.,"나는 내가 이것을할 수 있다고 확신하고 있었지만, 경우에만 나는 최고의 테니스를 칠..."


# 2. BERT-BASE 모델 학습


In [4]:
import pandas as pd
import torch
from transformers import BertModel, BertTokenizer, AdamW
from sklearn.model_selection import train_test_split

# BERT 모델 및 토크나이저 로드
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 데이터 전처리
def preprocess_data(df):
    inputs = tokenizer(df["sentence1"].tolist(), df["sentence2"].tolist(), return_tensors="pt", padding=True, truncation=True)
    labels = torch.tensor(df["label"].tolist(), dtype=torch.float32)
    return inputs, labels

# 데이터 나누기
train_data, eval_data = train_test_split(df, test_size=0.2, random_state=42)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 설정
device = torch.device("cpu")
model.to(device)
model.train()

# 학습 루프
num_epochs = 3
batch_size = 8

for epoch in range(num_epochs):
    # 미니배치 생성 및 학습
    for i in range(0, len(train_data), batch_size):
        batch = train_data.iloc[i:i+batch_size]
        inputs, labels = preprocess_data(batch)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)
        # 모델 학습 단계

        # Forward pass(순전파)
        outputs = model(**inputs)
        sentence_embeddings = outputs.last_hidden_state[:, 0, :]

        # Loss(손실률 계산)
        loss = torch.nn.functional.mse_loss(sentence_embeddings, labels.view(-1, 1))

        # 모델 조정 단계(손실을 최소화하는 과정)
        # Backward pass 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
# 모델 평가
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():  # Autograd 비활성화
    for i in range(0, len(eval_data), batch_size):
        batch = eval_data.iloc[i:i+batch_size]
        inputs, labels = preprocess_data(batch)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        # Forward pass
        outputs = model(**inputs)
        sentence_embeddings = outputs.last_hidden_state[:, 0, :]

        # 모든 예측값과 레이블을 리스트에 추가
        all_preds.extend(sentence_embeddings.cpu().detach().numpy())
        all_labels.extend(labels.cpu().detach().numpy())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will a

In [5]:
# 예측값과 실제값을 텐서로 변환
predictions = torch.tensor(all_preds)
labels = torch.tensor(all_labels).view(-1, 1)

# Mean Squared Error(평균제곱오차) 계산
Bmse = torch.nn.functional.mse_loss(predictions, labels)
print(f"Mean Squared Error: {Bmse.item()}")

Mean Squared Error: 18.639095306396484


<ipython-input-5-3fb36d45eba0>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  predictions = torch.tensor(all_preds)
<ipython-input-5-3fb36d45eba0>:6: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2, 768])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  Bmse = torch.nn.functional.mse_loss(predictions, labels)


# 3. simcse를 적용시킨 BERT-BASE 모델 학습(sup)


In [6]:
import pandas as pd
import torch
from transformers import BertTokenizer, AdamW, BertModel
from sklearn.model_selection import train_test_split

# SimCSE 모델 정의
class SimCSEModel(torch.nn.Module):
    def __init__(self, bert_model):
        super(SimCSEModel, self).__init__()
        self.bert = bert_model
        self.linear = torch.nn.Linear(768, 768)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        projected_output = self.linear(pooled_output)
        return projected_output


# BERT 모델 및 토크나이저 로드
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

# SimCSE 모델 초기화
simcse_model = SimCSEModel(bert_model)

# 데이터 전처리
def preprocess_data(df):
    inputs = tokenizer(df["sentence1"].tolist(), df["sentence2"].tolist(), return_tensors="pt", padding=True, truncation=True)
    labels = torch.tensor(df["label"].tolist(), dtype=torch.float32)
    return inputs, labels

# 데이터 나누기
train_data, eval_data = train_test_split(df, test_size=0.2, random_state=42)

# 옵티마이저 설정
optimizer = AdamW(simcse_model.parameters(), lr=5e-5)

# 학습 설정
device = torch.device("cpu")
simcse_model.to(device)
simcse_model.train()

# 학습 루프
num_epochs = 3
batch_size = 8

for epoch in range(num_epochs):
    for i in range(0, len(train_data), batch_size):
        batch = train_data.iloc[i:i+batch_size]
        inputs, labels = preprocess_data(batch)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        # Forward pass(순전파)
        outputs = simcse_model(**inputs)

        # Loss
        loss = torch.nn.functional.mse_loss(outputs, labels.view(-1, 1))

        # Backward pass 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

all_preds = []
all_labels = []

with torch.no_grad():
    for i in range(0, len(eval_data), batch_size):
        batch = eval_data.iloc[i:i+batch_size]
        inputs, labels = preprocess_data(batch)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        # Forward pass(순전파)
        outputs = simcse_model(**inputs)

        # 모든 예측값과 레이블을 리스트에 추가
        all_preds.extend(outputs.cpu().detach().numpy())
        all_labels.extend(labels.cpu().detach().numpy())


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, ove

In [7]:
# 예측값과 실제값을 텐서로 변환
predictions = torch.tensor(all_preds)
labels = torch.tensor(all_labels).view(-1, 1)

# Mean Squared Error 계산
sbmse = torch.nn.functional.mse_loss(predictions, labels)
print(f"Mean Squared Error: {sbmse.item()}")

Mean Squared Error: 17.005983352661133


<ipython-input-7-fb3566741215>:6: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2, 768])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  sbmse = torch.nn.functional.mse_loss(predictions, labels)


# 4. KOBERT 모델 학습


In [8]:
!pip install kobert-transformers

In [9]:
import pandas as pd
import torch
from transformers import AdamW, BertModel, BertTokenizer
from kobert_transformers import get_kobert_model, get_distilkobert_model

# KoBERT 모델과 토크나이저 불러오기
kobert_model = get_kobert_model()
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")

# 데이터 전처리
def preprocess_data(df):
    inputs = tokenizer.batch_encode_plus(
        df["sentence1"].tolist(),
        df["sentence2"].tolist(),
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    labels = torch.tensor(df["label"].tolist(), dtype=torch.float32)
    return inputs, labels

# 데이터 나누기
train_data, eval_data = train_test_split(df, test_size=0.2, random_state=42)

# 옵티마이저 설정
optimizer = AdamW(simcse_model.parameters(), lr=5e-5)

# 학습 설정
device = torch.device("cpu")
simcse_model.to(device)
simcse_model.train()

# 학습 루프
num_epochs = 3
batch_size = 8

for epoch in range(num_epochs):
    for i in range(0, len(train_data), batch_size):
        batch = train_data.iloc[i:i+batch_size]
        inputs, labels = preprocess_data(batch)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        # Forward pass
        outputs = simcse_model(**inputs)

        # Loss
        loss = torch.nn.functional.mse_loss(outputs, labels.view(-1, 1))

        # Backward pass 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

all_preds = []
all_labels = []

with torch.no_grad():
    for i in range(0, len(eval_data), batch_size):
        batch = eval_data.iloc[i:i+batch_size]
        inputs, labels = preprocess_data(batch)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        # Forward pass
        outputs = simcse_model(**inputs)

        # 모든 예측값과 레이블을 리스트에 추가
        all_preds.extend(outputs.cpu().detach().numpy())
        all_labels.extend(labels.cpu().detach().numpy())


<ipython-input-9-ae54cd020e9e>:48: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([5, 768])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = torch.nn.functional.mse_loss(outputs, labels.view(-1, 1))


In [10]:
# 예측값과 실제값을 텐서로 변환
predictions = torch.tensor(all_preds)
labels = torch.tensor(all_labels).view(-1, 1)

# Mean Squared Error 계산
kmse = torch.nn.functional.mse_loss(predictions, labels)
print(f"Mean Squared Error: {kmse.item()}")

Mean Squared Error: 16.622398376464844


<ipython-input-10-1375650eb05f>:6: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2, 768])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  kmse = torch.nn.functional.mse_loss(predictions, labels)


# 5. SIMCSE를 적용한 KOBERT 모델 학습


In [12]:
import pandas as pd
import torch
from transformers import AdamW, BertModel, BertTokenizer
from kobert_transformers import get_kobert_model, get_distilkobert_model

# KoBERT 모델과 토크나이저 불러오기
kobert_model = get_kobert_model()
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")

# SimCSE 모델 정의
class SimCSEModel(torch.nn.Module):
    def __init__(self, bert_model):
        super(SimCSEModel, self).__init__()
        self.bert = bert_model
        self.linear = torch.nn.Linear(768, 768)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        projected_output = self.linear(pooled_output)
        return projected_output



# SimCSE 모델 초기화
simcse_model = SimCSEModel(kobert_model)

# 데이터 전처리
def preprocess_data(df):
    inputs = tokenizer.batch_encode_plus(
        df["sentence1"].tolist(),
        df["sentence2"].tolist(),
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    labels = torch.tensor(df["label"].tolist(), dtype=torch.float32)
    return inputs, labels

# 데이터 나누기
train_data, eval_data = train_test_split(df, test_size=0.2, random_state=42)

# 옵티마이저 설정
optimizer = AdamW(simcse_model.parameters(), lr=5e-5)

# 학습 설정
device = torch.device("cpu")
simcse_model.to(device)
simcse_model.train()

# 학습 루프
num_epochs = 3
batch_size = 8

for epoch in range(num_epochs):
    for i in range(0, len(train_data), batch_size):
        batch = train_data.iloc[i:i+batch_size]
        inputs, labels = preprocess_data(batch)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        # Forward pass
        outputs = simcse_model(**inputs)

        # Loss
        loss = torch.nn.functional.mse_loss(outputs, labels.view(-1, 1))


        # Backward pass 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

all_preds = []
all_labels = []

with torch.no_grad():
    for i in range(0, len(eval_data), batch_size):
        batch = eval_data.iloc[i:i+batch_size]
        inputs, labels = preprocess_data(batch)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        # Forward pass
        outputs = simcse_model(**inputs)

        # Add all predictions and labels to lists
        all_preds.extend(outputs.cpu().detach().numpy())
        all_labels.extend(labels.cpu().detach().numpy())


<ipython-input-12-4c71a5e4d552>:66: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([5, 768])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = torch.nn.functional.mse_loss(outputs, labels.view(-1, 1))


In [13]:
# 예측값과 실제값을 텐서로 변환
predictions = torch.tensor(all_preds)
labels = torch.tensor(all_labels).view(-1, 1)

# Mean Squared Error 계산
ksmse = torch.nn.functional.mse_loss(predictions, labels)
print(f"Mean Squared Error: {ksmse.item()}")

Mean Squared Error: 17.629135131835938


<ipython-input-13-e34cd5f685c2>:6: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2, 768])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  ksmse = torch.nn.functional.mse_loss(predictions, labels)


# 6. 성능비교
KOBERT > simcse 적용한 BERT > simcse 적용한 KOBERT > BERT

In [14]:
import pandas as pd

variables = ['Bmse', 'sbmse', 'Kmse', 'Ksmse']
values = [Bmse.item(), sbmse.item(), kmse.item(), ksmse.item()]

data = {'변수': variables, '값': values}
df = pd.DataFrame(data)

print(df)

      변수          값
0   Bmse  18.639095
1  sbmse  17.005983
2   Kmse  16.622398
3  Ksmse  17.629135
